In [1]:
import os
import pickle
import sys


import numpy as np
import pandas as pd
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import tqdm
from torch.nn import functional as fnn
from torch.utils import data
from torch.utils.data import DataLoader
from torchvision import transforms

In [2]:
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

In [24]:
# Random seeds
np.random.seed(42)
torch.manual_seed(42)


TRAIN_SIZE = 0.9
BATCH_SIZE = 16
LEARNING_RATE = 1e-4
EPOCHS = 100
NUM_PTS = 971
CROP_SIZE = 224
SUBMISSION_HEADER = "file_name,Point_M0_X,Point_M0_Y,Point_M1_X,Point_M1_Y,Point_M2_X,Point_M2_Y,Point_M3_X,Point_M3_Y,Point_M4_X,Point_M4_Y,Point_M5_X,Point_M5_Y,Point_M6_X,Point_M6_Y,Point_M7_X,Point_M7_Y,Point_M8_X,Point_M8_Y,Point_M9_X,Point_M9_Y,Point_M10_X,Point_M10_Y,Point_M11_X,Point_M11_Y,Point_M12_X,Point_M12_Y,Point_M13_X,Point_M13_Y,Point_M14_X,Point_M14_Y,Point_M15_X,Point_M15_Y,Point_M16_X,Point_M16_Y,Point_M17_X,Point_M17_Y,Point_M18_X,Point_M18_Y,Point_M19_X,Point_M19_Y,Point_M20_X,Point_M20_Y,Point_M21_X,Point_M21_Y,Point_M22_X,Point_M22_Y,Point_M23_X,Point_M23_Y,Point_M24_X,Point_M24_Y,Point_M25_X,Point_M25_Y,Point_M26_X,Point_M26_Y,Point_M27_X,Point_M27_Y,Point_M28_X,Point_M28_Y,Point_M29_X,Point_M29_Y\n"

## Utils

In [4]:
class ScaleMinSideToSize(object):
    def __init__(self, size=(CROP_SIZE, CROP_SIZE), elem_name='image'):
        self.size = np.asarray(size, dtype=np.float)
        self.elem_name = elem_name

    def __call__(self, sample):
        h, w, _ = sample[self.elem_name].shape
        if h > w:
            f = self.size[0] / w
        else:
            f = self.size[1] / h

        sample[self.elem_name] = cv2.resize(sample[self.elem_name], None, fx=f, fy=f, interpolation=cv2.INTER_AREA)
        sample["scale_coef"] = f

        if 'landmarks' in sample:
            landmarks = sample['landmarks'].reshape(-1, 2).float()
            landmarks = landmarks * f
            sample['landmarks'] = landmarks.reshape(-1)

        return sample

In [5]:
class TransformByKeys(object):
    def __init__(self, transform, names):
        self.transform = transform
        self.names = set(names)

    def __call__(self, sample):
        for name in self.names:
            if name in sample:
                sample[name] = self.transform(sample[name])

        return sample

In [6]:
class CropCenter(object):
    def __init__(self, size=128, elem_name='image'):
        self.size = size
        self.elem_name = elem_name

    def __call__(self, sample):
        img = sample[self.elem_name]
        h, w, _ = img.shape
        margin_h = (h - self.size) // 2
        margin_w = (w - self.size) // 2
        sample[self.elem_name] = img[margin_h:margin_h + self.size, margin_w:margin_w + self.size]
        sample["crop_margin_x"] = margin_w
        sample["crop_margin_y"] = margin_h

        if 'landmarks' in sample:
            landmarks = sample['landmarks'].reshape(-1, 2)
            landmarks -= torch.tensor((margin_w, margin_h), dtype=landmarks.dtype)[None, :]
            sample['landmarks'] = landmarks.reshape(-1)

        return sample

In [7]:
class ThousandLandmarksDataset(data.Dataset):
    def __init__(self, root, transforms, split="train"):
        super(ThousandLandmarksDataset, self).__init__()
        self.root = root
        landmark_file_name = os.path.join(root, 'landmarks.csv') if split != "test" \
            else os.path.join(root, "test_points.csv")
        images_root = os.path.join(root, "images")

        self.image_names = []
        self.landmarks = []

        with open(landmark_file_name, "rt") as fp:
            num_lines = sum(1 for line in fp)
        num_lines -= 1  # header

        with open(landmark_file_name, "rt") as fp:
            for i, line in tqdm.tqdm(enumerate(fp), total=num_lines + 1):
                if i == 0:
                    continue  # skip header
                if split == "train" and i == int(TRAIN_SIZE * num_lines):
                    break  # reached end of train part of data
                elif split == "val" and i < int(TRAIN_SIZE * num_lines):
                    continue  # has not reached start of val part of data
                elements = line.strip().split("\t")
                image_name = os.path.join(images_root, elements[0])
                self.image_names.append(image_name)

                if split in ("train", "val"):
                    landmarks = list(map(np.int, elements[1:]))
                    landmarks = np.array(landmarks, dtype=np.int).reshape((len(landmarks) // 2, 2))
                    self.landmarks.append(landmarks)

        if split in ("train", "val"):
            self.landmarks = torch.as_tensor(self.landmarks)
        else:
            self.landmarks = None

        self.transforms = transforms

    def __getitem__(self, idx):
        sample = {}
        if self.landmarks is not None:
            landmarks = self.landmarks[idx]
            sample["landmarks"] = landmarks

        image = cv2.imread(self.image_names[idx])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        sample["image"] = image

        if self.transforms is not None:
            sample = self.transforms(sample)

        return sample

    def __len__(self):
        return len(self.image_names)


In [8]:
def restore_landmarks(landmarks, f, margins):
    dx, dy = margins
    landmarks[:, 0] += dx
    landmarks[:, 1] += dy
    landmarks /= f
    return landmarks

In [9]:
def restore_landmarks_batch(landmarks, fs, margins_x, margins_y):
    landmarks[:, :, 0] += margins_x[:, None]
    landmarks[:, :, 1] += margins_y[:, None]
    landmarks /= fs[:, None, None]
    return landmarks

## Train

In [10]:
def train(model, loader, loss_fn, optimizer, device):
    model.train()
    train_loss = []
    for batch in tqdm.tqdm(loader, total=len(loader), desc="Training..."):
        images = batch["image"].to(device) # B * 3 * CROP_SIZE * CROP_SIZE
        landmarks = batch["landmarks"] # B * (2 * NUM_PTS)
        
        pred_landmarks = model(images).cpu()
        loss = loss_fn(pred_landmarks, landmarks, reduction="mean")
        train_loss.append(loss.item())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    return np.mean(train_loss)

In [11]:
def validate(model, loader, loss_fn, device):
    model.eval()
    val_loss = []
    for batch in tqdm.tqdm(loader, total=len(loader), desc="validation..."):
        images = batch["image"].to(device)
        landmarks = batch["landmarks"]

        with torch.no_grad():
            pred_landmarks = model(images).cpu()
        loss = loss_fn(pred_landmarks, landmarks, reduction="mean")
        val_loss.append(loss.item())

    return np.mean(val_loss)

In [12]:
def predict(model, loader, device):
    model.eval()
    predictions = np.zeros((len(loader.dataset), NUM_PTS, 2))
    for i, batch in enumerate(tqdm.tqdm(loader, total=len(loader), desc="test prediction...")):
        images = batch["image"].to(device)

        with torch.no_grad():
            pred_landmarks = model(images).cpu()
        pred_landmarks = pred_landmarks.numpy().reshape((len(pred_landmarks), NUM_PTS, 2))  # B x NUM_PTS x 2

        fs = batch["scale_coef"].numpy()  # B
        margins_x = batch["crop_margin_x"].numpy()  # B
        margins_y = batch["crop_margin_y"].numpy()  # B
        prediction = restore_landmarks_batch(pred_landmarks, fs, margins_x, margins_y)  # B x NUM_PTS x 2
        predictions[i * loader.batch_size: (i + 1) * loader.batch_size] = prediction

    return predictions

## Main

In [13]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [14]:
device

device(type='cuda', index=0)

In [25]:
# 1. Define transforms and load data
train_transforms = transforms.Compose([
    ScaleMinSideToSize((CROP_SIZE, CROP_SIZE)),
    CropCenter(CROP_SIZE),
    TransformByKeys(transforms.ToPILImage(), ("image",)),
    TransformByKeys(transforms.ToTensor(), ("image",)),
    TransformByKeys(transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.25, 0.25, 0.25]), ("image",))
])

print("Data loading...")
train_dataset = ThousandLandmarksDataset("C:/Users/krvet/Desktop/MADE/CV/contest01_data/train/", train_transforms, split="train")
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=0, pin_memory=True, shuffle=True, drop_last=True)

val_dataset = ThousandLandmarksDataset("C:/Users/krvet/Desktop/MADE/CV/contest01_data/train/", train_transforms, split="val")
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, num_workers=0, pin_memory=True, shuffle=False, drop_last=False)



Data loading...


100%|██████████| 64001/64001 [00:02<00:00, 23454.41it/s]


In [28]:
class LandmarkModel(nn.Module):
    def __init__(self, pts=2 * NUM_PTS):
        super(LandmarkModel, self).__init__()
        self.model = models.resnext50_32x4d(pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.in_features, pts)
        self.fc_1 = nn.Linear(pts, 25 * pts)
        self.fc_2 = nn.Linear(25 * pts, pts)
        
    def forward(self, x):
        x = self.model(x)
        x = self.fc_1(x)
        x = self.fc_2(x)
        
        return x

In [29]:
print("Model creation")
model = LandmarkModel()

model.to(device)

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, amsgrad=True)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer)
loss_fn = fnn.mse_loss

#model.fc = nn.Linear(model.fc.in_features, 2 * NUM_PTS, bias=True)
#model.fc.requires_grad_(True)



Model creation


In [30]:
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

NVIDIA GeForce RTX 2070
Memory Usage:
Allocated: 0.8 GB
Cached:    5.2 GB


In [31]:
# 2. Train and validation
print("Train network")
best_val_loss = np.inf

for epoch in range(EPOCHS):
    train_loss = train(model, train_dataloader, loss_fn, optimizer, device=device)
    val_loss = validate(model, val_dataloader, loss_fn, device=device)
    scheduler.step(val_loss)
    print("Epoch #{:2}:\ttrain loss: {:5.2}\tval loss: {:5.2}".format(epoch + 1, train_loss, val_loss))
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        with open(os.path.join("./", "landmarks_best_resnext50_32x4d.pth"), "wb") as fp:
                torch.save(model.state_dict(), fp)

Training...:   0%|          | 0/3599 [00:00<?, ?it/s]

Train network


validation...: 100%|██████████| 401/401 [00:39<00:00, 10.15it/s]


Epoch # 1:	train loss: 8.8e+01	val loss: 2e+01


validation...: 100%|██████████| 401/401 [00:37<00:00, 10.71it/s]


...

Epoch # 7:	train loss:   7.9	val loss:   8.1


Training...:   0%|          | 1/3599 [00:00<08:11,  7.32it/s]

Epoch # 8:	train loss:   7.3	val loss:   8.4


Training...:  58%|█████▊    | 2096/3599 [09:44<06:58,  3.59it/s]


KeyboardInterrupt: 

In [34]:
# 3. Predict
test_dataset = ThousandLandmarksDataset("C:/Users/krvet/Desktop/MADE/CV/contest01_data/test", train_transforms, split="test")
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=0, pin_memory=True,
                                 shuffle=False, drop_last=False)

100%|██████████| 16001/16001 [00:00<00:00, 258775.08it/s]


In [ ]:
with open("./landmarks_best_resnext50_32x4d.pth", "rb") as fp:
        best_state_dict = torch.load(fp, map_location="cpu")
        model.load_state_dict(best_state_dict)

In [35]:


test_predictions = predict(model, test_dataloader, device=device)

test prediction...: 100%|██████████| 1000/1000 [01:49<00:00,  9.16it/s]


In [33]:
test_predictions

NameError: name 'test_predictions' is not defined

In [24]:
with open("./test_predictions.pkl", "wb") as fp:
    pickle.dump({"image_names": test_dataset.image_names,
                     "landmarks": test_predictions}, fp)

In [25]:
def create_submission(path_to_data, test_predictions, path_to_submission_file):
    test_dir = os.path.join(path_to_data, "test")

    output_file = path_to_submission_file
    wf = open(output_file, 'w')
    wf.write(SUBMISSION_HEADER)

    mapping_path = os.path.join(test_dir, 'test_points.csv')
    mapping = pd.read_csv(mapping_path, delimiter='\t')

    for i, row in mapping.iterrows():
        file_name = row[0]
        point_index_list = np.array(eval(row[1]))
        points_for_image = test_predictions[i]
        needed_points = points_for_image[point_index_list].astype(np.int)
        wf.write(file_name + ',' + ','.join(map(str, needed_points.reshape(2 * len(point_index_list)))) + '\n')

In [26]:
create_submission("C:/Users/krvet/Desktop/MADE/CV/contest01_data", test_predictions, "submit4.csv")